<img src="../data/images/dl_logos.jpg" width="600">

# Curriculum
* keras library
* data loading and preprocessing 
* build simple logistic regression model
* add updateable plot 
* build first neural network
* play with the cool stuff: activations, drop-out, stochastic gradient descent
* visualize the receptive fields


# import modules and set plot environment

In [ ]:
%pylab inline
import keras
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

check keras version

In [ ]:
keras.__version__


# data loading

In [ ]:
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [ ]:
Y_test

# data preprocessing

In [ ]:
from keras.utils import to_categorical

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.

In [ ]:
Y_test

In [ ]:
def plotMNIST(X,Y,  maxN=30, print_digits=True):
    # plot pixels
    plt.gray()
    fig = plt.figure( figsize=(16,7) )
    for i in range(0,maxN):
        if print_digits:
            ax = fig.add_subplot(3,10,i+1, title='Digit: ' + str(Y[i,:].argmax()) )
        else:
            ax = fig.add_subplot(3,10,i+1)
        
        ax.matshow(X[i,:].reshape((28,28)).astype(float))
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
plotMNIST(X_train, Y_train, 30)




# plain logistic regression, to keep it simple and fast
<img src="../data/images/logreg.png" width="800">
<img src="../data/images/sigmoid.png" width="400">

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

logreg = Sequential()

logreg.add(Flatten(input_shape=(28, 28, 1)))
logreg.add(Dense(10))
logreg.add(Activation('softmax'))

logreg.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# fit the model

In [ ]:
logreg.fit(X_train, Y_train,
          epochs=10,
          validation_data=(X_test, Y_test),
          verbose=1)

# make updatable plot to show training progress

In [ ]:
class PlotLossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        #self.fig = plt.subplots(figsize=(16, 8), ncols=2)
        #self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        plt.subplots(figsize=(16, 8), ncols=2)
        clear_output(wait=True)
        plt.subplot(221)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.subplot(222)
        plt.plot(self.x, self.acc, label="accuracy")
        plt.plot(self.x, self.val_acc, label="val_accuracy")
        plt.legend()
        plt.show();
        
ShowLoss = PlotLossHistory()

In [ ]:
logreg = Sequential()

logreg.add(Flatten(input_shape=(28, 28, 1)))
logreg.add(Dense(10))
logreg.add(Activation('softmax'))
logreg.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
logreg.fit(X_train, Y_train,
          epochs=10,
          validation_data=(X_test, Y_test),
          callbacks=[ShowLoss],
          verbose=1)

# plot receptive fields

In [ ]:
plotMNIST(logreg.get_weights()[0].T, Y_train, 10,print_digits=False)

# make predictions

In [ ]:
label = logreg.predict(X_test[0:50,:])

In [ ]:
plotMNIST(X_test, label, 30)

# simple two layer neural network
<img src="../data/images/nnet2.png" width="800">
<img src="../data/images/sigmoid.png" width="400">

In [ ]:
NNet = Sequential()

NNet.add(Flatten(input_shape=(28, 28, 1)))
NNet.add(Dense(50,  activation='sigmoid'))
NNet.add(Dense(10))
NNet.add(Activation('softmax'))
NNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
NNet.fit(X_train, Y_train,
          epochs=25,
          validation_data=(X_test, Y_test),
          callbacks=[ShowLoss],
          verbose=1)

# changing the activation functions

<img src="../data/images/activation.PNG" width="600">

In [ ]:
NNet = Sequential()

NNet.add(Flatten(input_shape=(28, 28, 1)))
NNet.add(Dense(50,  activation='relu'))
NNet.add(Dense(10))
NNet.add(Activation('softmax'))
NNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
NNet.fit(X_train, Y_train,
          epochs=25,
          validation_data=(X_test, Y_test),
          callbacks=[ShowLoss],
          verbose=1)

# model regulariation with dropout
<img src="../data/images/dropout.png" width="800">


In [ ]:
from keras.layers import Flatten, Dense, Activation, Dropout

In [ ]:
NNet = Sequential()

NNet.add(Flatten(input_shape=(28, 28, 1)))
NNet.add(Dense(50,  activation='relu'))
NNet.add(Dropout(0.25))
NNet.add(Dense(10))
NNet.add(Activation('softmax'))
NNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
NNet.fit(X_train, Y_train,
          epochs=25,
          validation_data=(X_test, Y_test),
          callbacks=[ShowLoss],
          verbose=1)

# model regulariation by stochastic gradient descent

In [ ]:
NNet = Sequential()
NNet.add(Flatten(input_shape=(28, 28, 1)))
NNet.add(Dense(50,  activation='relu'))
NNet.add(Dropout(0.25))
NNet.add(Dense(10))
NNet.add(Activation('softmax'))
NNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
NNet.fit(X_train, Y_train,
          epochs=25,
          validation_data=(X_test, Y_test),
          callbacks=[ShowLoss],
          verbose=1, batch_size=64)

# unsupervised learning - autoencoders
<img src="../data/images/autoencoder.jpg" width="800">

In [ ]:
from keras.layers import Input
from keras.models import Model


x_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
x_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))


encoding_dim = 32 
input_img = Input(shape=(784,))
encoded = Dropout(0.5)(input_img)
encoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(784, activation='sigmoid')(encoded)


autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


class PlotAELossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        plt.subplots(figsize=(8, 4), ncols=1)
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
  
        plt.legend()
        plt.show();
        
ShowAELoss = PlotAELossHistory()

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                callbacks=[ShowAELoss],
                validation_data=(x_test, x_test))

# build encoder and decoder models

In [ ]:
encoder = Model(input_img, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# enode and reconstruct some test data

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# plot the reconstruction

In [ ]:
plotMNIST(x_test, label, 10,print_digits=False)
plotMNIST(decoded_imgs, label, 10,print_digits=False)